Insert explanatory text here. Allow 30 minutes for completion.

In [1]:
load("preamble.sage")

Construct the quadratic relations cutting out the orthogonal Grassmannian $OG$.

In [2]:
F = GF(2)
P.<x0,x12,x13,x14,x15,x23,x24,x25,x34,x35,x45,x1234,x1235,x1245,x1345,x2345> = PolynomialRing(F, 16)
quads = [x0*x2345 + x23*x45 + x24*x35 + x25*x34,
         x12*x1345 + x13*x1245 + x14*x1235 + x15*x1234,
         x0*x1345 + x13*x45 + x14*x35 + x15*x34,
         x12*x2345 + x23*x1245 + x24*x1235 + x25*x1234,
         x0*x1245 + x12*x45 + x14*x25 + x15*x24,
         x13*x2345 + x23*x1345 + x34*x1235 + x35*x1234,
         x0*x1235 + x12*x35 + x13*x25 + x15*x23,
         x14*x2345 + x24*x1345 + x34*x1245 + x45*x1234,
         x0*x1234 + x12*x34 + x13*x24 + x14*x23,
         x15*x2345 + x25*x1345 + x35*x1245 + x45*x1235]

In [3]:
def linear_section(coords):
    V = Matrix(coords).right_kernel()
    return [sum(P.gens()[i] * v[-1-i] for i in range(16)) for v in V.gens()] + quads

Read in data about 6-tuples of points on $OG$.

In [4]:
with open("6-tuples.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
    l = [[vector(F, v) for v in vecs] for vecs in l]
len(l)

494

For each of these tuples, compute the dimension of the linear span, and tabulate the effect on the linear span of adding any one $\mathbb{F}_2$-rational point.

In [5]:
proj = ProjectiveSpace(P)
OG = proj.subscheme(quads)
rp = [vector(pt) for pt in OG.rational_points()]
len(rp)

2295

In [6]:
d = {}
l2a = []
l2b = []
V0 = VectorSpace(F, 16)
for vecs in l:
    M0 = Matrix(F, vecs).echelon_form()
    M0.set_immutable()
    if M0.rank() == 6:
        l2a.append(vecs)
    elif M0.rank() == 5:
        l2b.append(vecs)
    else:
        raise AssertionError
    d[M0] = []
    for (v,) in subspaces_containing(V0, V0.subspace(vecs), 1, basis=True):
        M = Matrix(F, vecs + [v]).echelon_form()
        M.set_immutable()
        d[M0].append(M)
    for pt in rp:
        if pt not in vecs:
            M = Matrix(F, vecs + [pt]).echelon_form()
            M.set_immutable()
            if M in d[M0]:
                d[M0].remove(M)
len(l2a), len(l2b)

(491, 3)

In cases where the linear span is 6-dimensional, identify 7-dimensional extensions which do not pick up any more $\mathbb{F}_2$-points.

In [7]:
l3a = []
for vecs in l2a:
    M0 = Matrix(F, vecs).echelon_form()
    M0.set_immutable()
    for M in d[M0]:
        l3a.append(M.rows())
len(l3a)

76916

In cases where the linear span is 5-dimensional, identify 6-dimensional extensions whose intersection with $OG$ is zero-dimensional with no extra degree-1 points.

In [8]:
for vecs in l2b:
    M0 = Matrix(F, vecs).echelon_form()
    M0.set_immutable()
    for M in d[M0][:]:
        J = P.ideal(linear_section(M.rows()))
        if J.dimension() > 1:
            d[M0].remove(M)
            continue

In cases where the linear span is 5-dimensional, identify options for pairs of generators which do not add any $\mathbb{F}_2$-points to the linear span.

In [9]:
l3b = []
for vecs in l2b:
    M0 = Matrix(F, vecs).echelon_form()
    M0.set_immutable()
    b = []
    d2 = defaultdict(int)
    for M1, M2 in itertools.combinations(d[M0], 2):
        M = Matrix(F, M1.rows() + M2.rows()).echelon_form()
        M.set_immutable()
        d2[M] += 1
    for M in d2:
        assert d2[M] <= 3
        if d2[M] == 3:
            l3b.append(M)
len(l3b)

101503

In [10]:
del d, d2

Enforce the condition that $\#C(\mathbb{F}_4) = 18$.

In [11]:
def count_by_ideal(vecs, n):
    gens = linear_section(vecs)
    J = P.ideal(gens + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [12]:
l4a = [vecs for vecs in l3a if count_by_ideal(vecs, 2) == 18]
len(l4a)

503

In [13]:
l4b = [vecs for vecs in l3b if count_by_ideal(vecs, 2) == 18]
len(l4b)

576

Merge the two lists, then enforce the condition that $\#C(\mathbb{F}_8) = 12$.

In [ ]:
l5 = [vecs for vecs in l4a+l4b if count_by_ideal(vecs, 3) == 12]
len(l5)

Use Magma to compute the full zeta functions of the resulting curves and compare these to our targets, finding no results.

In [ ]:
proj_magma = magma.ProjectiveSpace(P)
l6 = []
for vecs in l5:
    gens = linear_section(vecs)
    X = proj_magma.Scheme(gens)
    C = X.Curve()
    if str(X.IsIrreducible()) == "false" or C.Genus() != 7:
        continue
    ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 8))
    if ct in targets7:
        l6.append(C)
len(l6)

In [ ]:
l6 = isomorphism_class_reps(l6)
len(l6)

Search for extensions of relative class number 1.

In [ ]:
l7 = []
for C in l6:
    F0 = C.FunctionField()
    F1 = F0.AlgorithmicFunctionField()
    l7 += match_weil_poly(F1, 2)
assert not l7

In [ ]:
print("This stage complete!")